In [80]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [81]:
data = pd.read_excel("Dry_Bean_Dataset.xlsx")
data

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653248,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [82]:
data.isnull().sum()

Area               0
Perimeter          0
MajorAxisLength    0
MinorAxisLength    0
AspectRation       0
Eccentricity       0
ConvexArea         0
EquivDiameter      0
Extent             0
Solidity           0
roundness          0
Compactness        0
ShapeFactor1       0
ShapeFactor2       0
ShapeFactor3       0
ShapeFactor4       0
Class              0
dtype: int64

### Handle categorical data

In [83]:
X = data.drop(columns='Class')

target = data['Class'].values
target = data['Class'].factorize()[0]
target

array([0, 0, 0, ..., 6, 6, 6], dtype=int64)

### initilize papulation

In [84]:
def initialize_population(data, number_of_cluster=7, population_size=75):
    population = []
    for _ in range(population_size):
        centroids = data[np.random.choice(data.shape[0], number_of_cluster, replace=False)]
        population.append(centroids)
    return population


### Fitness function

In [85]:
def fitness_function(data, centroids,  truelabels): 
    distances = np.zeros((data.shape[0], centroids.shape[0]))
    for i in range(centroids.shape[0]):
        distances[:, i] = np.linalg.norm(data - centroids[i], axis=1)
    cluster_labels = np.argmin(distances, axis=1)
    fitness = (truelabels == cluster_labels).sum
    return fitness


### crossover function 

In [86]:
import random
def crossover(parent1, parent2):
    child1, child2 = np.copy(parent1) , np.copy(parent2)
    for _ in range(len(child1)):
        x1 , x2 = random.randint(0, 2), random.randint(0, 2)

        child1[_] = parent1[_] if x1 == 0 else  parent2[_]
        child2[_] = parent2[_] if x2 == 0 else parent1[_]
        

    return child1, child2

### mutation function

In [87]:
def mutate(centroid, mutation_rate, data):
    if np.random.rand() < mutation_rate:
        random_point = data[np.random.randint(0, data.shape[0])]
        centroid[np.random.randint(0, len(centroid))] = random_point
    return centroid 

### implement genetic algorithm

In [88]:
def genetic_algorithm(data, k, population_size, num_generations, mutation_rate):
    population = initialize_population(data, k, population_size)

    for generation in range(num_generations):
        
        fitnesses = np.array([fitness_function(centroids, data, target) for centroids in population])
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = np.random.choice(population, size=2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate, data)
            child2 = mutate(child2, mutation_rate, data)
            new_population.extend([child1, child2])
            print("best fitness score: ", max(fitnesses))
        population = new_population
    best_solution = max(population, key=lambda centroids: fitness_function(centroids, data, target))
    return best_solution

In [89]:
best_centroids = genetic_algorithm(data, k=7, population_size=20, num_generations=10, mutation_rate=0.01)


KeyError: "None of [Index([8240, 12171, 329, 6739, 9601, 1652, 7483], dtype='int32')] are in the [columns]"